In [1]:
%pip install xgboost


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import xgboost as xgb
print("XGBoost imported successfully!")

XGBoost imported successfully!


In [3]:
import sys
print(sys.executable)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import xgboost as xgb
import warnings

warnings.filterwarnings(action='ignore', category=UserWarning, message='.*gpu_id.*')
warnings.filterwarnings(action='ignore', category=UserWarning, message='.*gpu_hist.*')

file_path = r"/home/innobridge/sherry/Final_Balanced_Data.csv"
data = pd.read_csv(file_path)

poverty_threshold = 13590
data['poverty_status'] = np.where(data['PINCP'] <= poverty_threshold, 1, 0)

data = data.drop(columns=['PINCP'])

X = data.drop(columns=['poverty_status'])
Y = data['poverty_status']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=18)

param_grid = {
    'n_estimators': [400, 500, 600],             
    'max_depth': [7, 8, 9],   
    'learning_rate': [0.08, 0.1],   
    'min_child_weight': [2, 3],
  'gamma': [0], 
    'subsample': [1.0],
    'colsample_bytree': [0.6],
    'colsample_bylevel': [1.0],
    'colsample_bynode': [1.0],
   'reg_alpha': [10],
    'reg_lambda': [1],
    'max_delta_step': [0],
    'scale_pos_weight': [1]
}

xgb_model = xgb.XGBClassifier(random_state=18, eval_metric='logloss')

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X_train, Y_train)

best_xgb_model = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Make predictions with the best model on the training set
train_predictions = best_xgb_model.predict(X_train)

# Calculate evaluation metrics for the training set
train_accuracy = accuracy_score(Y_train, train_predictions)
train_precision = precision_score(Y_train, train_predictions)
train_recall = recall_score(Y_train, train_predictions)
train_f1 = f1_score(Y_train, train_predictions)
train_conf_matrix = confusion_matrix(Y_train, train_predictions)

print("Training Metrics:")
print(f"Accuracy: {train_accuracy}")
print(f"Precision: {train_precision}")
print(f"Recall: {train_recall}")
print(f"F1 Score: {train_f1}")
print(f"Confusion Matrix: \n{train_conf_matrix}")

# Make predictions with the best model on the testing set
predictions = best_xgb_model.predict(X_test)

# Calculate evaluation metrics for the testing set
accuracy = accuracy_score(Y_test, predictions)
precision = precision_score(Y_test, predictions)
recall = recall_score(Y_test, predictions)
f1 = f1_score(Y_test, predictions)
conf_matrix = confusion_matrix(Y_test, predictions)

print("Testing Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix: \n{conf_matrix}")

/home/innobridge/anaconda3/bin/python
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters found:  {'colsample_bylevel': 1.0, 'colsample_bynode': 1.0, 'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.08, 'max_delta_step': 0, 'max_depth': 9, 'min_child_weight': 2, 'n_estimators': 500, 'reg_alpha': 10, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1.0}
Training Metrics:
Accuracy: 0.8269618309622628
Precision: 0.8422206264640123
Recall: 0.8049347714678743
F1 Score: 0.8231556880905461
Confusion Matrix: 
[[282939  50316]
 [ 65088 268585]]
Testing Metrics:
Accuracy: 0.8162020487968716
Precision: 0.8304803141677575
Recall: 0.7934389167478384
F1 Score: 0.8115371606039175
Confusion Matrix: 
[[70107 13468]
 [17177 65980]]
